# Project SPIKE: a study on spiking neural networks
Rodrigo Pereira Cruz <br>
pereirarodrigocs@gmail.com <br>
<br>
***

> This Jupyter notebook is used for studying spiking neural networks, more specifically the integrate-and-fire and leaky integrate-and-fire models. It is focused on its applications in machine learning, reinforcement learning and its overall impact on computational intelligence. As stated in the readme file, this project will make heavy use of BindsNET for neural simulations.

## 1. Brief introduction to neuronal dynamics
***
Neurons can be divided into three different parts: dendrites, soma and axon. The dendrites, roughly speaking, act as the neuron's input receivers and relay this information directly to the neuron's body, the soma, which essentially acts as a processing unit: should the processed input exceed a certain threshold, then an output signal is generated and transmitted to the axon, which relays the output to other neurons. The junction between two neurons is known as the synapse - the terms presynaptic and postsynaptic originate from this definition, and are often used to designate a neuron that transmits information and a neuron that receives it, respectively.

<br>

<center>

![Neuron](https://neuronaldynamics.epfl.ch/online/x3.png)

<br>

<font size = "2"> **Fig. 1**: The overall structure of a neuron, with information being transmitted between them (represented by the arrows). Here, we have $j$ as the presynaptic neuron and $i$ as the postsynaptic one. Source: https://neuronaldynamics.epfl.ch/online/x3.png. </font>

<br>

</center>

<br>


Thus, in a neuronal system, neurons must be able to communicate with each other all the time. This communication, which is an electrical process, happens through action potentials, an event that is driven by a sum of changes in the membrane potential, which generate a spike when crossing a stimulation threshold.

<br>

<center>

![Neuron action potential](https://neuronaldynamics.epfl.ch/online/x9.png)

<br>

<font size = "2"> **Fig. 2**: Two sequential presynaptic neuron spikes, received as input in a postsynaptic neuron, cause two postsynaptic potentials that become a cumulative action potential. Note that the threshold $\vartheta$ is not reached, thus a spike is not produced. Source: https://neuronaldynamics.epfl.ch/online/x9.png. </font>

<br>

</center>

<br>

Consider a presynaptic neuron $j = 1$ and a postsynaptic neuron $i$. Neuron $j = 1$ fires spikes at $t_1^{(1)}, t_1^{(2)}, ...$. Each of those spikes evokes a postsynaptic potential $\epsilon_{i1}$, which is defined, mathematically, as

<br>

$$u_i(t) = \sum_j \epsilon_{i1}(t - t_1^{(j)}) + u_{rest}$$

<br>

where $u_{rest}$ corresponds to the neuron's at rest state value, which is $\approx \mbox{-65 } mV$.

<br>

Should there be more spikes, such that $j = 1, 2$ for example, the total change of the potential takes a cumulative linear form, such that

<br>

$$u_i(t) = \sum_j \sum_f \epsilon_{ij}(t - t_j^{(f)}) + u_{rest}$$

<br>

so long as there are only a few input spikes.


#2. Integrate-and-fire models

In [ ]:
# Installing BindsNET
#!pip install git+https://github.com/BindsNET/bindsnet.git

from bindsnet.network import Network